In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [32]:
!wget https://drive.google.com/file/d/1h8At-lChKzM8mPf0lDF1P9a3PzpociVg/view?usp=sharing

--2025-02-16 06:38:17--  https://drive.google.com/file/d/1h8At-lChKzM8mPf0lDF1P9a3PzpociVg/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 64.233.170.139, 64.233.170.101, 64.233.170.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.170.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing.1’

view?usp=sharing.1      [ <=>                ]  93.09K  --.-KB/s    in 0.03s   

2025-02-16 06:38:18 (3.09 MB/s) - ‘view?usp=sharing.1’ saved [95324]



In [33]:
data = pd.read_csv('davis.txt', header = None, delimiter=' ')

data.rename(columns={0: 'drug_id', 1: 'protein_id', 2: 'SMILES', 3: 'Protein', 4: 'pKb'}, inplace=True)
data.head()

,drug_id,protein_id,SMILES,Protein,pKb
0,11314340,AAK1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.366532
1,11314340,ABL1(E255K),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
2,11314340,ABL1(F317I),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
3,11314340,ABL1(F317I)p,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
4,11314340,ABL1(F317L),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000


In [34]:
max_smiles_length = 100
size_vocab_smiles = 0
max_protein_length = 1000
size_vocab_protein = 0

In [35]:
!pip install rdkit

In [36]:
# encoding data

from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def tokenize_smiles(smiles):
    """Tokenize a SMILES string into individual characters."""
    tokens = []
    for char in smiles:
        tokens.append(char)
    return tokens

# Build vocabulary
def build_vocab(smiles_list):
    vocab = set()
    for smiles in smiles_list:
        vocab.update(tokenize_smiles(smiles))
    vocab = {char: idx for idx, char in enumerate(sorted(vocab))}
    vocab["[UNK]"] = len(vocab)  # Add special [UNK] token
    return vocab

# Apply tokenization and mapping
def smiles_to_indices(smiles, vocab, max_length):
    tokenized = tokenize_smiles(smiles)
    indices = [vocab.get(char, vocab["[UNK]"]) for char in tokenized]  # Use [UNK] for unknown tokens
    return indices[:max_length] + [0] * (max_length - len(indices))



In [37]:
# Amino acids
amino_acids = "ACDEFGHIKLMNPQRSTVWY"
aa_vocab = {aa: idx + 1 for idx, aa in enumerate(amino_acids)}
aa_vocab["[PAD]"] = 0
aa_vocab["X"] = len(amino_acids) + 1

size_vocab_protein = len(aa_vocab)

def protein_to_indices(protein, vocab, max_length):
    """Convert protein sequence into indices."""
    indices = [vocab.get(aa, vocab["X"]) for aa in protein]
    return indices[:max_length] + [0] * (max_length - len(indices))


In [38]:
# Split data such that each protein has a fixed ratio of entries in both training and testing split

def split_by_protein(data):
    # Group data by protein
    grouped = data.groupby('protein_id')

    train_indices = []
    test_indices = []

    # Split each group
    for _, group in grouped:
        train_idx, test_idx = train_test_split(group.index, test_size=0.3, random_state=42)
        train_indices.extend(train_idx)
        test_indices.extend(test_idx)

    # Create train and test datasets
    train_dataframe = data.iloc[train_indices, :]
    # train_dataset = torch.utils.data.TensorDataset(X[train_indices], y[train_indices])
    test_dataframe = data.iloc[test_indices, :]
    # test_dataset = torch.utils.data.TensorDataset(X[test_indices], y[test_indices])
    return train_dataframe, test_dataframe

In [39]:
# Split data that some proteins are entirely new in the testing split

def split_new_proteins(data):
    # Get unique protein IDs
    unique_proteins = data['protein_id'].unique()

    # Shuffle the protein IDs
    np.random.shuffle(unique_proteins)

    # Split the protein IDs into training and testing sets
    train_proteins = unique_proteins[:int(0.9 * len(unique_proteins))]
    test_proteins = unique_proteins[int(0.9 * len(unique_proteins)):]

    # Get the indices for the training and testing sets
    train_indices = data[data['protein_id'].isin(train_proteins)].index
    test_indices = data[data['protein_id'].isin(test_proteins)].index

    # Create train and test datasets
    # train_datasframe = torch.utils.data.TensorDataset(X[train_indices], y[train_indices])
    # test_dataset = torch.utils.data.TensorDataset(X[test_indices], y[test_indices])
    train_dataframe = data.iloc[train_indices, :]
    test_dataframe = data.iloc[test_indices, :]
    return train_dataframe, test_dataframe

In [40]:
# Split data such that each drug has a fixed ratio of entries in both training and testing split

def split_by_drug(data):
    # Group data by drug
    grouped = data.groupby('drug_id')

    train_indices = []
    test_indices = []

    # Split each group
    for _, group in grouped:
        train_idx, test_idx = train_test_split(group.index, test_size=0.3, random_state=42)
        train_indices.extend(train_idx)
        test_indices.extend(test_idx)

    # Create train and test datasets
    # train_dataset = torch.utils.data.TensorDataset(X[train_indices], y[train_indices])
    # test_dataset = torch.utils.data.TensorDataset(X[test_indices], y[test_indices])
    train_dataframe = data.iloc[train_indices, :]
    test_dataframe = data.iloc[test_indices, :]
    return train_dataframe, test_dataframe

In [41]:
# Split data that some drugs are entirely new in the testing split

def split_new_drugs(data):
    # Get unique drug IDs
    unique_drugs = data['drug_id'].unique()

    # Shuffle the drug IDs
    np.random.shuffle(unique_drugs)

    # Split the drug IDs into training and testing sets
    train_drugs = unique_drugs[:int(0.9 * len(unique_drugs))]
    test_drugs = unique_drugs[int(0.9 * len(unique_drugs)):]

    # Get the indices for the training and testing sets
    train_indices = data[data['drug_id'].isin(train_drugs)].index
    test_indices = data[data['drug_id'].isin(test_drugs)].index

    # Create train and test datasets
    # train_dataset = torch.utils.data.TensorDataset(X[train_indices], y[train_indices])
    # test_dataset = torch.utils.data.TensorDataset(X[test_indices], y[test_indices])
    train_dataframe = data.iloc[train_indices, :]
    test_dataframe = data.iloc[test_indices, :]
    return train_dataframe, test_dataframe

In [42]:
# temporary splitting

# X_smiles_train, X_smiles_test, X_protein_train, X_protein_test, y_train, y_test = train_test_split(
#     X_smiles, X_protein, y_pKb, test_size=0.2, random_state=42
# )

In [68]:
data_train, data_test = split_new_drugs(data)

# SMILES
# train
smiles_list_train = data_train['SMILES'].to_list()
vocab = build_vocab(smiles_list_train)
size_vocab_smiles = len(vocab)
smiles_indices_train = [smiles_to_indices(sm, vocab, max_length=max_smiles_length) for sm in smiles_list_train]
# test
smiles_list_test = data_test['SMILES'].to_list()
vocab = build_vocab(smiles_list_test)
size_vocab_smiles = len(vocab)
smiles_indices_test = [smiles_to_indices(sm, vocab, max_length=max_smiles_length) for sm in smiles_list_test]

# Proteins
proteins_list_train = data_train['Protein'].to_list()
protein_indices_train = [protein_to_indices(protein_seq, aa_vocab, max_length=1000) for protein_seq in proteins_list_train]
# test
proteins_list_test = data_test['Protein'].to_list()
protein_indices_test = [protein_to_indices(protein_seq, aa_vocab, max_length=1000) for protein_seq in proteins_list_test]


In [69]:
# Convert to model-ready arrays

# train
X_smiles_train = np.array(smiles_indices_train)
X_protein_train = np.array(protein_indices_train)
y_train = data_train["pKb"].values

# test
X_smiles_test = np.array(smiles_indices_test)
X_protein_test = np.array(protein_indices_test)
y_test = data_test["pKb"].values

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout, Concatenate
)

# SMILES Encoder
smiles_input = Input(shape=(max_smiles_length,), name="smiles_input")
smiles_embedding = Embedding(input_dim=size_vocab_smiles, output_dim=128)(smiles_input)
smiles_lstm = LSTM(units=256, return_sequences=False)(smiles_embedding)

# Protein Sequence Encoder
protein_input = Input(shape=(max_protein_length,), name="protein_input")
protein_embedding = Embedding(input_dim=size_vocab_protein, output_dim=128)(protein_input)
protein_lstm = LSTM(units=256, return_sequences=False)(protein_embedding)

# Combine Drug and Protein Representations
combined = Concatenate()([smiles_lstm, protein_lstm])
dense1 = Dense(128, activation="relu")(combined)
dropout1 = Dropout(0.3)(dense1)
dense2 = Dense(64, activation="relu")(dropout1)
output = Dense(1, activation="linear", name="pKb_output")(dense2)

# Build Model
model = Model(inputs=[smiles_input, protein_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ smiles_input (InputLayer) │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ protein_input             │ (None, 1000)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_10 (Embedding)  │ (None, 100, 128)       │          2,432 │ smiles_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_11 (Embedding)  │ (None, 1000, 128)      │          2,816 │ protein_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_10 (LSTM)            │ (None, 256)            │        394,240 │ embedding_10[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_11 (LSTM)            │ (None, 256)            │        394,240 │ embedding_11[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 512)            │              0 │ lstm_10[0][0],         │
│ (Concatenate)             │                        │                │ lstm_11[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 128)            │         65,664 │ concatenate_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 128)            │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 64)             │          8,256 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pKb_output (Dense)        │ (None, 1)              │             65 │ dense_11[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 867,713 (3.31 MB)

 Trainable params: 867,713 (3.31 MB)

 Non-trainable params: 0 (0.00 B)

In [71]:
# Batch size and number of epochs
batch_size = 32
epochs = 20

# Train the model
history = model.fit(
    [X_smiles_train, X_protein_train],  # Inputs: SMILES and Protein sequences
    y_train,                           # Labels
    validation_split=0.1,              # Use 10% of training data for validation
    batch_size=batch_size,
    epochs=epochs,
    verbose=1                         # Print training progress
)

Epoch 1/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 52s 64ms/step - loss: 2.1368 - mae: 0.9577 - val_loss: 1.5369 - val_mae: 0.9381
Epoch 2/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 47s 62ms/step - loss: 0.8824 - mae: 0.6641 - val_loss: 2.1723 - val_mae: 1.2317
Epoch 3/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 84s 64ms/step - loss: 0.8503 - mae: 0.6465 - val_loss: 2.7880 - val_mae: 1.4598
Epoch 4/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 82s 63ms/step - loss: 0.7893 - mae: 0.6253 - val_loss: 2.6242 - val_mae: 1.4041
Epoch 5/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - loss: 0.8163 - mae: 0.6341 - val_loss: 3.0991 - val_mae: 1.5646
Epoch 6/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 48s 63ms/step - loss: 0.7808 - mae: 0.6245 - val_loss: 2.4567 - val_mae: 1.3471
Epoch 7/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - loss: 0.7699 - mae: 0.6179 - val_loss: 2.2314 - val_mae: 1.2658
Epoch 8/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - loss: 0.7499 - mae: 0.5977 - val_loss: 2.9925 - val_mae: 1.5422
Epoch 9/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 

In [72]:
!pip install lifelines

In [73]:
from lifelines.utils import concordance_index
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import Binarizer
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Concordance index
def evaluate_ci(test_targets, test_predictions):
    # Calculate the concordance index
    c_index = concordance_index(test_targets, test_predictions)
    print(f'Concordance Index: {c_index}')

# Mean square error
def evaluate_mse(test_targets, test_predictions):
    # Calculate the mean-square error
    mse = mean_squared_error(test_targets, test_predictions)
    print(f'Mean-Square Error: {mse}')

# Pearson correlation coefficient
def evaluate_pearsonr(test_targets, test_predictions):
    # Calculate the Pearson correlation coefficient
    pearson_corr, _ = pearsonr(test_targets, test_predictions)
    print(f'Pearson Correlation Coefficient: {pearson_corr}')

# Area under the precision-recall curve
def evaluate_auprc(test_targets, test_predictions):
    # Binarize the targets and predictions
    binarizer = Binarizer(threshold=7.0)
    test_targets_bin = binarizer.fit_transform(np.array(test_targets).reshape(-1, 1)).flatten()
    test_predictions_bin = binarizer.transform(np.array(test_predictions).reshape(-1, 1)).flatten()

    precision, recall, _ = precision_recall_curve(test_targets_bin, test_predictions_bin)
    auprc = auc(recall, precision)
    print(f'Area Under the Precision-Recall Curve: {auprc}')

# r_m^2 index
def evaluate_rm2(test_targets, test_predictions):
    y_obs = np.array(test_targets)
    y_pred = np.array(test_predictions)

    # Calculate r^2
    r2 = r2_score(y_obs, y_pred)

    # Calculate r_0^2 (Linear regression without intercept)
    model_no_intercept = LinearRegression(fit_intercept=False)
    model_no_intercept.fit(y_obs.reshape(-1, 1), y_pred)
    y_pred_no_intercept = model_no_intercept.predict(y_obs.reshape(-1, 1))
    r2_0 = r2_score(y_obs, y_pred_no_intercept)

    if (r2 - r2_0 < 0):
        print(f'r^2: {r2}, r_0^2: {r2_0}')
        print('r_m^2 index is not defined')
        return

    # Calculate r_m^2
    rm2 = r2 * (1 - np.sqrt(r2 - r2_0))
    print(f'rm2: {rm2}')

In [74]:
# Predict on the test set
y_pred = model.predict([X_smiles_test, X_protein_test])

y_test = np.array(y_test, dtype=np.float64).flatten()
y_test = np.nan_to_num(y_test, nan=np.nanmean(y_test))
y_pred = np.array(y_pred, dtype=np.float64).flatten()
y_pred = np.nan_to_num(y_pred, nan=np.nanmean(y_pred))

# Evaluate the model
evaluate_ci(y_test, y_pred)
evaluate_mse(y_test, y_pred)
evaluate_pearsonr(y_test, y_pred)
evaluate_auprc(y_test, y_pred)
evaluate_rm2(y_test, y_pred)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step
Concordance Index: 0.3247884570647624
Mean-Square Error: 2.4899075827481987
Pearson Correlation Coefficient: -0.16331547169434454
Area Under the Precision-Recall Curve: 0.5505817711700065
r^2: -1.5908692091681491, r_0^2: -0.6465549540362863
r_m^2 index is not defined
